In [2]:
# Imports
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection.search import GridSearchCV
from surprise import CoClustering
from surprise.prediction_algorithms import predictions

%load_ext autoreload
%autoreload 2

In [3]:
from helpersCoClustering import*

# Load dataset and samples into a pandas data frame
DATA_TRAIN_PATH = 'data/data_train.csv'
data_np = load_data(DATA_TRAIN_PATH)


DATA_TEST_PATH = 'data/sampleSubmission.csv'
samples = load_data(DATA_TEST_PATH)

In [4]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(data_np[['user_id', 'movie_id', 'rating']], reader=reader)

## Basic Co-Clustering
ID : 25590
- RMSE : 1.050
- Secondary: 0.118

In [20]:
trainset = data.build_full_trainset()
basic_cocltr = CoClustering().fit(trainset)

## Parameters test
ID : 25685
- RMSE : 1.042
- Secondary: 0.131

In [67]:
trainset = data.build_full_trainset()
algo = CoClustering(n_cltr_u=2, n_cltr_i=19).fit(trainset)

## Grid Search

In [32]:
param_grid = {'n_cltr_u': [2], 'n_cltr_i': [12, 16, 17, 18, 19]}

gs = GridSearchCV(CoClustering, param_grid, measures=['rmse'], cv=3, n_jobs = 5)

In [33]:
# Do the grid search
gs.fit(data)

In [34]:
# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

1.0060522318449017
{'n_cltr_u': 2, 'n_cltr_i': 19}


## Create submission

In [36]:
samples['rating'] = samples[['user_id', 'movie_id']] \
.apply(lambda row: algo.predict(row['user_id'], row['movie_id'])[3], axis=1)

samples.head()

,user_id,movie_id,rating
0,37,1,3.318467
1,73,1,3.007555
2,156,1,3.790951
3,160,1,3.159863
4,248,1,3.488879


In [37]:
samples['rating'] = samples['rating'].apply(round)

In [38]:
PATHOUT = "Predictions.csv"
create_csv(PATHOUT,samples)